# Recursive Feature Addition by `Mr. Harshit Dawar!`
* This method can be used to remove the Features either on the basis of the Regression Coefficients or Tree Importance!

* It will first of all train the model on all the features, then calculate the metric score on which the model has to be evaluated.

* A Threshold/Tolerance level is defined, if the difference between the Initial Prediction & the prediction after adding a feature recursively is greater than the Tolerance Level, then the Feature will be kept & added to the list of the Features to keep from the Dataset, otherwise it is removed.

In [1]:
# Importing the required Libraries!

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xgboo
from sklearn.ensemble import XGB
from sklearn.metrics import  accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder